In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
df_green= spark.read.parquet('/home/Дмитрий/datacamp/dataeng-zoomcamp/week_5_batch_processing/data/pq/green/*/*')

```
select 
    -- Reveneue grouping 
    date_trunc('hour', lpep_pickup_datetime) as hour,
    PULocationID as zone,

    -- Revenue calculation 
    sum(total_amount) as amount,
    count(1) as number_records

    from green
    where lpep_pickup_datetime >= '2020-01-01'
    group by 1,2
```

In [ ]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [ ]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [ ]:
rows = rdd.take(10)
row = rows[0]
row

In [ ]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [ ]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [ ]:
from collections import namedtuple

In [ ]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [ ]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [ ]:
from pyspark.sql import types

In [ ]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [ ]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) \
    .show()

In [ ]:
df_result.write.parquet('/home/Дмитрий/datacamp/dataeng-zoomcamp/week_5_batch_processing/data/tmp/green-revenue')